In [2]:
#import libraries
import requests
from bs4 import BeautifulSoup
import json
import numpy as np
import pandas as pd
import re 
import openpyxl
import time
#from requests.exceptions import HTTPError

In [76]:
def replace_nan(ar):
    for i in range(len(ar)):
        if ar[i]==None:
            ar[i]=0
    return ar

def del_substr(x):
    clean=list()
    for u in range(len(x)):
        string=re.sub('/>\n[0-9]','',x[u]['description'])
        string=re.sub('\r<br','',string)
        string=re.sub('&quot;','',string)
        clean.append(re.sub('/>\n','',string))
    return clean

In [63]:
#request parameters
s=requests.Session()
url = 'https://api.hh.ru/vacancies'
params={'text':('"data scientist" OR "аналитик данных"')} #precise search

In [64]:
#downlod all urls from all pages
request_=s.get(url, params=params,stream=True).json()
n_pages=request_['pages']
n_vac=request_['per_page']
download_urls=list()
for i in range(n_pages+1):
    params={'text':('"data scientist" OR "аналитик данных"'),'page':i}
    pages=s.get(url, params=params,stream=True).json()
    download_urls.extend([u['alternate_url'] for u in pages['items']])

download_urls=[download_urls[j][-8:] for j in range(len(download_urls))] #getll vacancy IDs

In [65]:
vac=list()
link='https://api.hh.ru/vacancies/{}'
try:
    for url in range(len(download_urls)):
        vacancy_desc=s.get(link.format(url)).json()
        if vacancy_desc['description']!='Not Found':
            vac.append(vacancy_desc)
except KeyboardInterrupt:
    pass

In [138]:
#get list of all previous vacancies
url_list=list()
try:
    file=pd.read_excel('data scientist vacancies.xlsx')
    url_list=list(file['URL'])
except (FileNotFoundError, OSError):
    print('File has not been found')

In [139]:
try:
    wb = openpyxl.load_workbook('data scientist vacancies.xlsx',data_only=True)
    sheet = wb.active
except (FileNotFoundError, OSError):
    print('File has not been found')

In [140]:
try:
    while True:
        #get response
        url = 'https://api.hh.ru/vacancies'
        params={'text':('"data scientist" OR "аналитик данных"'),'page':1}
        pages=s.get(url, params=params,stream=True).json()
        download_urls.extend([u['alternate_url'] for u in pages['items']])
        download_urls=[download_urls[j][-8:] for j in range(len(download_urls))]
        
        vac=list()
        link='https://api.hh.ru/vacancies/{}'
        for url in range(len(download_urls)):
            vacancy_desc=s.get(link.format(url)).json()
            if vacancy_desc['description']!='Not Found':
                vac.append(vacancy_desc)
        #get responsibilities
        responsib=del_substr(vac)
        #get salary
        salary=list()
        for i in range(len(vac)):
            if vac[i]['salary']!=None:
                salary.append('from {} to {}, currency {}'.format(vac[i]['salary']['from'],vac[i]['salary']['to'],vac[i]['salary']['currency']))
        #get name and URL
            
        vac_name=[vac[n]['name'] for n in range(len(vac))]
        urls=[vac[n]['alternate_url'] for n in range(len(vac))]
            
        #create dataframe
        data=pd.concat([pd.Series(urls),pd.Series(vac_name), pd.Series(salary), pd.Series(responsib)],axis=1)
        data.columns=['URL','Name','Salary','Responsibilities']
        data=data.reset_index(drop=True)

        #append to excel file, given vacancies have not been added yet
        for i in range(len(data.iloc[:,0])):
            if data.iloc[i,0] not in url_list: 
                sheet.append(list(data.iloc[i,:]))
                url_list.append(data.iloc[i,0])

        time.sleep(30)
       # else:
        #    print('Connection error has occured: {}'.format(page.status_code))
except KeyboardInterrupt:
    pass

In [141]:
try:
    wb.save('data scientist vacancies.xlsx')
    print('Workbook saved')
except PermissionError:
    print ('Close the workbook and try again')

Workbook saved


In [129]:
responsib=del_substr(vac)
        #get salary
salary=list()
for i in range(len(vac)):
    if vac[i]['salary']!=None:
        salary.append('from {} to {}, currency {}'.format(vac[i]['salary']['from'],vac[i]['salary']['to'],vac[i]['salary']['currency']))
        #get name and URL
            
vac_name=[vac[n]['name'] for n in range(len(vac))]
urls=[vac[n]['alternate_url'] for n in range(len(vac))]
            
        #create dataframe
data=pd.concat([pd.Series(urls),pd.Series(vac_name), pd.Series(salary), pd.Series(responsib)],axis=1)
data.columns=['URL','Name','Salary','Responsibilities']
data=data.reset_index(drop=True)

In [130]:
data

,URL,Name,Salary,Responsibilities
0,https://hh.ru/vacancy/3,EVENT MANAGER,"from None to 2000, currency USD",Срочно приглашается EVENT MANAGER в крупную з...
1,https://hh.ru/vacancy/5,Курьер,"from None to 200, currency USD",Одна из ведущих коммуникационных групп на росс...
2,https://hh.ru/vacancy/6,Менеджер отдела развития и маркетинга,"from None to 2500, currency USD",Компания Бест была основана в 1992 году. На на...
3,https://hh.ru/vacancy/7,МЕНЕДЖЕР ПО КАНАЛЬНОМУ МАРКЕТИНГУ,"from None to 500, currency USD",Обязанности: . Планирование совместно с отдел...
4,https://hh.ru/vacancy/9,Экономист (бухгалтер) отдела обучения SAP,"from None to 800, currency USD",Обязанности: . Подготовка статистики по обуче...
5,https://hh.ru/vacancy/10,Юрист пресс-службы,"from None to 400, currency USD",Обязанности: Работа в пресс-службе крупной ком...
6,https://hh.ru/vacancy/12,Экономист (бухгалтер) отдела обучения SAP,"from None to 1200, currency USD",Обязанности: . Подготовка статистики по обуче...
7,https://hh.ru/vacancy/13,Медиа-менеджер,"from None to 1500, currency USD",В известное РА полного цикла. Требования: Опы...
8,https://hh.ru/vacancy/14,"Бухгалтер по экспорту. Код вакансии: ""Н-СБЭ""","from None to 800, currency USD",В крупную нефтяную компанию приглашается бухга...
9,https://hh.ru/vacancy/15,Менеджер по рекламе,"from None to 1200, currency USD","Рекламное агентство полного цикла, входящее в..."
